In [ ]:
!pip install torch transformers datasets scikit-learn matplotlib seaborn


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")


In [ ]:
print(dataset)


In [ ]:
print(dataset["train"][0:5])


In [ ]:
print("Train size:", len(dataset["train"]))
print("Test size:", len(dataset["test"]))


In [ ]:
labels = dataset["train"]["label"]
print(set(labels))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

label_counts = Counter(labels)

plt.figure()
sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()))
plt.title("Class Distribution")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()


From the class distribution plot, we observe that some sentiment classes have more samples than others. The dataset is slightly imbalanced but not extremely skewed

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(set(labels))
)


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],   # this will now be a list when batched=True
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [ ]:
def tokenize_function(example):
    # Replace None or non-string values with empty string
    texts = [
        str(t) if t is not None else ""
        for t in example["text"]
    ]

    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [ ]:
dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
dataset = dataset.remove_columns(["id", "text", "sentiment"])


In [ ]:
dataset.set_format("torch")


In [ ]:
print(dataset["train"][0])


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset["train"], batch_size=16, shuffle=True)
test_loader = DataLoader(dataset["test"], batch_size=16)


In [ ]:
from transformers import BertForSequenceClassification
import torch

num_labels = len(set(dataset["train"]["label"]))

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Using device:", device)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


In [ ]:
epochs = 2

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:

        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Loss: {avg_loss}")


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


From the confusion matrix, we observe that most predictions lie on the diagonal, indicating correct classifications. Some misclassification occurs between class 0 and class 1, which may be due to similarity in sentiment

In [ ]:
original_dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

label_names = {}

for example in original_dataset["train"]:
    label_names[example["label"]] = example["sentiment"]

print(label_names)


In [ ]:
import torch.nn.functional as F

def predict_text(text: str):
    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = F.softmax(outputs.logits, dim=1)
        confidence, predicted_class = torch.max(probs, dim=1)

    label = predicted_class.item()

    return {
        "Predicted Label": label_names[label],
        "Confidence": float(confidence.item())
    }


In [ ]:
predict_text("I absolutely loved this movie!")
predict_text("This was the worst product ever.")
predict_text("It was fine, nothing special.")


In [ ]:
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")


In this assignment, we fine-tuned a pre-trained BERT-base-uncased model for multiclass sentiment classification using PyTorch.
The model achieved an accuracy of approximately 76–77% on the test set.
The results demonstrate the effectiveness of transfer learning for text classification tasks